<P>The below is some code I wrote as Data Analyst to understand how entities were differently distributed across events within different sites</p>

In [ ]:
import numpy as np
import pandas as pd
import vertica_python as vp
import datetime as dt
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Command so that plots appear in the iPython Notebook
%matplotlib inline

In [63]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 500;

<IPython.core.display.Javascript object>

In [31]:
import vertica_python as vp
%run /Users/eric.abis/Documents/credential.py

In [82]:
sql_string = """ 



        SELECT display_name, entity_id, 
                --pct_of_events, 
                SUM(pct_of_events) OVER (PARTITION BY site_id  ORDER BY site_id, pct_of_events desc RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as pct_events_cum_sum,
                --pct_of_properties,                        
                SUM(pct_of_properties) OVER (PARTITION BY site_id  ORDER BY site_id, pct_of_events desc RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as pct_prop_cum_sum,
                SUM(1) OVER (PARTITION BY site_id  ORDER BY site_id, pct_of_events desc RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as count_of_properties
        FROM
                (
                SELECT site_id, entity_id, 
                (count(*)/SUM(count(*)) OVER (PARTITION BY site_id)) * 100 as pct_of_events,  100/count(entity_id) OVER (PARTITION BY site_id) as pct_of_properties
                FROM company_log_data_production.events a
                WHERE
                    ad_unit_type = 'PRODUCT_1' 
                    AND a.Filter_Field_1 = false 
                    AND Filter_Field_2 = 'FILTER_VALUE' 
                GROUP BY 1,2
                HAVING count(*) > 0
                ORDER BY 1,3 DESC
                ) main
                
                INNER JOIN company.sites s on s.id = site_id 
        ORDER BY display_name, pct_of_events desc;
                


            """

print 'Pulling Data'

conn = vp.connect(**conn_info)
data = pd.read_sql_query(sql_string, conn)
data.head()

Pulling Data


In [186]:
copied = data[data["display_name"].isin([
"Entity_1",
"Entity_2",
"Entity_3",
"Entity_4",
"Entity_5",
"Entity_6",
"Entity_7"])].copy()

In [ ]:
# copied.set_index(["pct_prop_cum_sum", "count_of_properties"], inplace = True)
# dropped = data.drop(["display_name", "hotel_property_id", "pct_prop_cum_sum"], axis = 1)
grouped = copied.groupby("display_name")

fig, ax = plt.subplots(figsize = (20,15))

for site, results in grouped: 

    plt.tick_params(axis='both', labelsize=15)
    ax.plot(results["pct_prop_cum_sum"], 
            results["pct_ad_calls_cum_sum"],
           label = site)
    ax.set_xlabel("Percent of Properties", fontsize=25)
    ax.set_ylabel("Percent of Ad Calls", fontsize = 25)

    # copied.set_index(["pct_prop_cum_sum", "count_of_properties"], inplace = True)
    # dropped = data.drop(["display_name", "hotel_property_id", "pct_prop_cum_sum"], axis = 1)
ax.legend(loc = 'best', fontsize = 22, bbox_to_anchor=(-.05,1))
    
fig, ax = plt.subplots(figsize = (20,15))

for site, results in grouped:     
    
    plt.tick_params(axis='both', labelsize=15)
    ax.plot(results["count_of_properties"].values, 
            results["pct_ad_calls_cum_sum"],
            label = site)
    ax.set_xlabel("Count of Properties", fontsize=25)
    ax.set_ylabel("Percent of Ad Calls", fontsize = 25)
    
ax.legend(loc = 'best', fontsize = 22, bbox_to_anchor=(-.05,1))

<img src="pic1.png">

<img src="pic2.png">